# Tech Intro

## Main tables

In [8]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [9]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = real_estate_list[1] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [10]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

22/11/07 11:16:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/07 11:16:53 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/07 11:16:53 WARN Utils: Service 'SparkUI' could not bind on port 4080. Attempting port 4081.
22/11/07 11:16:53 WARN Utils: Service 'SparkUI' could not bind on port 4081. Attempting port 4082.
22/11/07 11:16:53 WARN Utils: Service 'SparkUI' could not bind on port 4082. Attempting port 4083.
22/11/07 11:16:53 WARN Utils: Service 'SparkUI' could not bind on port 4083. Attempting port 4084.
22/11/07 11:16:53 WARN Utils: Service 'SparkUI' could 

## Read the spec-file from Hadoop and show some data sample

In [11]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+--------+-----------------+----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------+-----------+---------------+--------+-----------+----------------------------------------+------+---------------------+----------+--------------------------------------------------------------------------------------------+-------+-------+-----+--------+------+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Show columns' properties

In [12]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- session: string (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- object_type: string (nullable = true)
 |-- object_sub_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- person: string (nullable = true)
 |-- p_type: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- address: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region2: string (nullable = true)
 |-- city2: string (nullable = true)
 |-- district: string (nullable = true)
 |-- street: string (nullable = true)
 |-- house: string (nullable = true)
 |-- description: string (nullable = true)
 |-- type: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- source: string (nullable = true)
 |-- location: string (nullable = true)
 |-- location2: string (nullable = true)
 |-- area: str

## Total rows (count)

In [13]:
eda_df.count()

10906778

## Simple func to execute and show the result of the SQL-instruction

In [14]:
def execute(select_instruction: str = "select count(*) from realEstate") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from realEstate"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show()

## Create the temp-table-view from the DF (called as "eda_sql")

In [15]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [16]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|10906778|
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [17]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,session,url,title,object_type,object_sub_type,price,phone,person,p_type,...,house_type,parking_type,ad_type,total_floors,is_new,is_apartment,metro,is_actual,object_type_str,pdf_link_hash
0,0,0,0,0,40682,6602926,0,1009,1,227,...,10484126,10851085,297372,8001665,10717615,6177448,5384899,0,4357562,0


In [18]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,session,url,title,object_type,object_sub_type,price,phone,person,p_type,...,house_type,parking_type,ad_type,total_floors,is_new,is_apartment,metro,is_actual,object_type_str,pdf_link_hash
0,0,0,0,0,0,0,14603,0,1496,0,...,0,0,0,278,137951,4624876,0,9707,0,0


In [19]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,session,url,title,object_type,object_sub_type,price,phone,person,p_type,...,house_type,parking_type,ad_type,total_floors,is_new,is_apartment,metro,is_actual,object_type_str,pdf_link_hash
0,0,0,1,20,0,0,0,0,29109,0,...,0,0,0,0,0,0,0,0,0,10906514


In [20]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
session,0,0,0
url,0,0,1
title,0,0,20
object_type,40682,0,0
object_sub_type,6602926,0,0
price,0,14603,0
phone,1009,0,0
person,1,1496,29109
p_type,227,0,0


In [21]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [22]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
session,0.000,0.000,0.000
url,0.000,0.000,0.000
title,0.000,0.000,0.000
object_type,0.373,0.000,0.000
object_sub_type,60.540,0.000,0.000
price,0.000,0.134,0.000
phone,0.009,0.000,0.000
person,0.000,0.014,0.267
p_type,0.002,0.000,0.000


### SQL-instructions

In [ ]:
#Find dupes or misunderstanding or useless static values 10 906 778 total
execute("SELECT house_area\
        FROM eda_sql\
        WHERE house_area < 9\
        GROUP BY house_area\
        ORDER BY house_area ASC\
        ")

execute("SELECT due_time\
        FROM eda_sql\
        GROUP BY due_time\
        ORDER BY due_time DESC\
        ")

execute("SELECT total_floors\
        FROM eda_sql\
        GROUP BY total_floors\
        ORDER BY total_floors ASC\
        ")

execute("SELECT object_type_str\
        FROM eda_sql\
        GROUP BY object_type_str\
        ORDER BY object_type_str DESC\
        ")

execute("SELECT metro\
        FROM eda_sql\
        GROUP BY metro\
        ORDER BY metro ASC\
        ")

execute("SELECT residential_complex_name, count(residential_complex_name)\
        FROM eda_sql\
        GROUP BY residential_complex_name\
        ORDER BY count(residential_complex_name) DESC\
        ")

execute("SELECT living_area, count(living_area)\
        FROM eda_sql\
        GROUP BY living_area\
        ORDER BY living_area DESC\
        ")

execute("SELECT land_category, count(land_category)\
        FROM eda_sql\
        GROUP BY land_category\
        ORDER BY count(land_category) DESC\
        ")

execute("SELECT ad_id, count(ad_id)\
        FROM eda_sql\
        GROUP BY ad_id\
        ORDER BY count(ad_id) DESC\
        ")

execute("SELECT id, area, description\
        FROM eda_sql\
        WHERE id = 13461558\
        ")

execute("SELECT location2, count(location2)\
        FROM eda_sql\
        GROUP BY location2\
        ORDER BY count(location2) DESC")

execute("SELECT id, location\
        FROM eda_sql\
        ORDER BY location")

execute("SELECT source, count(source)\
        FROM eda_sql\
        GROUP BY source\
        ORDER BY source ASC")

execute("SELECT source_id\
        FROM eda_sql\
        GROUP BY source_id\
        ORDER BY source_id ASC")

execute("SELECT source_id\
        FROM eda_sql\
        GROUP BY source_id\
        ORDER BY source_id DESC")

execute("SELECT type\
        FROM eda_sql\
        GROUP BY type\
        --WHERE type IS NOT NULL\
        --ORDER BY person DESC")

execute("SELECT id, description\
        FROM eda_sql\
        WHERE description LIKE 'x%'\
        --ORDER BY person DESC")

execute("SELECT id, description\
        FROM eda_sql\
        WHERE description LIKE '1'\
        --ORDER BY person DESC")

execute("SELECT id, person\
        FROM eda_sql\
        WHERE person LIKE '*%'\
        ORDER BY person DESC")

execute("SELECT id, person\
        FROM eda_sql\
        WHERE person LIKE '1%'\
        ORDER BY person DESC")

execute("SELECT id, phone\
        FROM eda_sql\
        WHERE phone >1\
        ORDER BY phone DESC")

execute("SELECT id, price\
        FROM eda_sql\
        WHERE price >30000000\
        ORDER BY price DESC")

execute("SELECT id, price\
        FROM eda_sql\
        WHERE price = 9 OR price = 99 OR price = 999\
        ORDER BY price asc")
execute("SELECT object_sub_type, count(object_sub_type)\
        FROM eda_sql\
        GROUP BY object_sub_type\
        --ORDER BY title DESC")

execute("SELECT id\
        , title\
        FROM eda_sql\
        WHERE title not like '1%'\
        ORDER BY title DESC")

execute("SELECT id\
        , title\
        FROM eda_sql\
        WHERE title not like ' %'\
        AND title IS NOT NULL\
        ORDER BY title ASC")

execute("SELECT id\
        , url\
        FROM eda_sql\
        WHERE url not like 'https%'\
        ORDER BY url DESC")

execute("SELECT id\
        , url\
        FROM eda_sql\
        WHERE url not like 'http%'\
        ORDER BY url DESC")

for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")


+----------+
|house_area|
+----------+
|       0.0|
|       1.0|
|       2.0|
|       2.1|
|       2.3|
|       2.5|
|       2.8|
|       3.0|
|      3.11|
|      3.15|
|       3.2|
|       3.4|
|       3.5|
|       3.6|
|      3.64|
|      3.71|
|       3.8|
|       4.0|
|      4.07|
|       4.2|
+----------+
only showing top 20 rows



+---------------+
|       due_time|
+---------------+
|           сдан|
|            кв.|
|     4 кв. 2031|
|     4 кв. 2029|
|4 кв. 2028 года|
|     4 кв. 2028|
|4 кв. 2027 года|
|     4 кв. 2027|
|     4 кв. 2026|
|4 кв. 2025 года|
|     4 кв. 2025|
|4 кв. 2024 года|
|     4 кв. 2024|
|4 кв. 2023 года|
|     4 кв. 2023|
|4 кв. 2022 года|
|     4 кв. 2022|
|4 кв. 2021 года|
|     4 кв. 2021|
|4 кв. 2020 года|
+---------------+
only showing top 20 rows



+------------+
|total_floors|
+------------+
|        null|
|           0|
|           1|
|          10|
|         100|
|         101|
|         102|
|         109|
|          11|
|         110|
|         113|
|         116|
|         117|
|         118|
|         119|
|          12|
|         120|
|         122|
|         123|
|          13|
+------------+
only showing top 20 rows



+--------------------+
|     object_type_str|
+--------------------+
|      часть коттеджа|
|          часть дома|
|  торговое помещение|
|            таунхаус|
| складское помещение|
|производственное ...|
|помещение свободн...|
|   офисное помещение|
|         новостройка|
|         машиноместо|
|             коттедж|
|коммерческая недв...|
|квартира, апартам...|
|   земельный участок|
|          дом, вилла|
|                 дом|
|                дача|
|           гостиница|
|  гараж, машиноместо|
|               гараж|
+--------------------+
only showing top 20 rows



+-------------------+
|              metro|
+-------------------+
|               null|
|   Электрозаводская|
|                  1|
|              1 мая|
|1 микрорайон ГПЗ-23|
|                1-й|
|         1-й ГПЗ-23|
|        1-й участок|
|                 10|
|             11, 12|
|               11-й|
|                11А|
|                11Б|
|               12-й|
|                 13|
|               13-й|
|       13-й квартал|
|                13А|
|                 14|
|               14-й|
+-------------------+
only showing top 20 rows



...

+---------------------------+
|count(DISTINCT object_type)|
+---------------------------+
|                         13|
+---------------------------+



+-------------------------------+
|count(DISTINCT object_sub_type)|
+-------------------------------+
|                             24|
+-------------------------------+



+---------------------+
|count(DISTINCT price)|
+---------------------+
|               692440|
+---------------------+



+---------------------+
|count(DISTINCT phone)|
+---------------------+
|              3920171|
+---------------------+



+----------------------+
|count(DISTINCT person)|
+----------------------+
|                245480|
+----------------------+



## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [7]:
spark.stop()

NameError: name 'spark' is not defined